In [1]:
from astropy.io import fits
from matplotlib import pyplot as plt
import numpy as np
import glob

In [3]:
path = '/Users/ha/Makani_Project/DrizzlePac'
lst_raw = glob.glob(path + '/data/*/*/*raw.fits')
dark_lst = glob.glob(path + '/darks/HST/*/*raw.fits')

In [11]:
w_data = np.zeros((1024,1024))
total_time = 0
uptemp = 25.0
lowtemp = 0.0
for item in dark_lst:
    file = fits.open(item)
    
    exptime = file[0].header['EXPTIME']
    temp = (file[1].header['MDECODT1'] + file[1].header['MDECODT2']) /2 
    year = int(file[0].header['DATE'][:4])
    data = file[1].data
    
    #if temp <= uptemp and year > 2010 and year != 2019 and year != 2020:
    #    w_data += data
    #    total_time += exptime
    
    if temp <= 22.0 and year > 2010:
        w_data += data
        total_time += exptime
    
w_data_cps = w_data / total_time
    
avg_dark_Primary = fits.PrimaryHDU(w_data_cps)
avg_dark_hdul = fits.HDUList([avg_dark_Primary])
avg_dark_hdul.writeto('avg_dark_uptemp22.fits', overwrite = True)

In [26]:
%time
dark_file = fits.open('avg_dark_uptemp25.fits')
sumdata_1subtracted = np.zeros((1024,1024))
total_exptime = 0
for i in range(len(lst_raw)):
    filename = lst_raw[i]
    rawfile = fits.open(filename)
    rawdata = rawfile[1].data
    exptime = rawfile[0].header['EXPTIME']
    
    dark_frame = dark_file[0].data * exptime
    new_data = rawdata - dark_frame

    Primary = fits.PrimaryHDU(rawdata)
    hdul = fits.HDUList([Primary])
    hdul.writeto(filename[-18:-9] + '_original.fits', overwrite = True)
    
    subtracted_Primary = fits.PrimaryHDU(new_data)
    subtracted_hdul = fits.HDUList([subtracted_Primary])
    subtracted_hdul.writeto(filename[-18:-9] + '_subtracted_25.fits', overwrite = True)\
    
    total_time += exptime
    sumdata_1subtracted += new_data
    
Primary = fits.PrimaryHDU(sumdata_1subtracted/total_time)
hdul = fits.HDUList([Primary])
hdul.writeto('sumdata_afterprimarysubt_ps.fits', overwrite = True)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 11 µs


In [33]:
print(sum(sum(sumdata_1subtracted/total_time))/ (1024 **2))

2.6471555845654753e-05


In [32]:
sumdark = np.zeros((1024,1024))
total_time = 0
for item in dark_lst:
    file = fits.open(item)
    
    exptime = file[0].header['EXPTIME']
    temp = (file[1].header['MDECODT1'] + file[1].header['MDECODT2']) /2 
    year = int(file[0].header['DATE'][:4])
    data = file[1].data
    
    #if temp <= uptemp and year > 2010 and year != 2019 and year != 2020:
    #    w_data += data
    #    total_time += exptime
    
    if year > 2021:
        sumdark += data - dark_file[0].data * exptime
        total_time += exptime
    
sumdark_cps = sumdark / total_time
print(total_time)
print(sum(sum(sumdark_cps))/ (1024 **2))
print(sum(sum(sumdark))/ (1024 **2))
sumdark_Primary = fits.PrimaryHDU(sumdark_cps)
sumdark_hdul = fits.HDUList([avg_dark_Primary])
sumdark_hdul.writeto('sumdark_25up.fits', overwrite = True)

24080.0
1.6508558568349717e-05
0.397526090325861
